<a href="https://colab.research.google.com/github/sandhrabijoy/_/blob/master/Pytorch_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#installing the necessary libraries
!pip install matplotlib tqdm torchmetrics torch torchvision

In [9]:
#importing libraries
import matplotlib.pyplot as plt
import torchmetrics
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms


In [10]:
#setting device and loading the data
device= torch.device("cuda" if torch.cuda.is_available()else "cpu")

#loading of data
transform=transforms.Compose([transforms.ToTensor(),  # Convert images to PyTorch tensors
                              transforms.Normalize((0.5,), (0.5,)) ]) # Normalize pixel values])

trainset=torchvision.datasets.MNIST(root='./data/',train=True,download=True,transform=transform)
testset=torchvision.datasets.MNIST(root='./data/',train=False, download=True,transform=transform)

In [11]:
#building dataset class
class MNISTDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.unsqueeze(1).float().to(device)  # Add channel dimension: [N, 1, 28, 28]
        self.y = F.one_hot(y, num_classes=10).float().to(device)  # One-hot encode labels

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [12]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("CUDA Device Name:", torch.cuda.get_device_name(0))
    print("CUDA Version:", torch.version.cuda)
else:
    print("CUDA is not available. Check your installation.")



CUDA Available: False
CUDA Device Count: 0
CUDA is not available. Check your installation.


In [13]:
#creating the dataloader
train_dataset=MNISTDataset(trainset.data,trainset.targets)
test_dataset=MNISTDataset(testset.data,testset.targets)

train_loader=DataLoader(train_dataset,shuffle=True,batch_size=32)
test_loader=DataLoader(test_dataset,shuffle=True,batch_size=32)

In [14]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)

        # Placeholder for fc1 input size (to be computed later)
        self.flattened_size = None

        # Fully connected layers
        self.fc1 = nn.Linear(1, 1)  # Placeholder (will update after computing dynamically)
        self.fc2 = nn.Linear(1200, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        # Dynamically compute the flattened size only once
        if self.flattened_size is None:
            self.flattened_size = x.view(x.size(0), -1).size(1)
            self.fc1 = nn.Linear(self.flattened_size, 1200).to(x.device)

        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x


In [15]:
model=MyModel().to(device)

In [16]:
#defining the parameters
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)
epochs=5

In [17]:
from tqdm import tqdm

print("tqdm is installed")


tqdm is installed


In [18]:
#training the model

for epoch in tqdm(range(epochs)):
  model.train()
  running_loss = 0.0
  for X_batch,y_batch in train_loader:
    optimizer.zero_grad()

    y_pred=model(X_batch)
    loss = loss_fn(y_pred, y_batch.argmax(dim=1))  # Convert one-hot labels to class indices

    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")


  print(f'Finished epoch{epoch},latest loss{loss}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.0105
Epoch 1/5, Loss: 0.0181
Epoch 1/5, Loss: 0.0258
Epoch 1/5, Loss: 0.0328
Epoch 1/5, Loss: 0.0385
Epoch 1/5, Loss: 0.0426
Epoch 1/5, Loss: 0.0462
Epoch 1/5, Loss: 0.0502
Epoch 1/5, Loss: 0.0531
Epoch 1/5, Loss: 0.0555
Epoch 1/5, Loss: 0.0581
Epoch 1/5, Loss: 0.0598
Epoch 1/5, Loss: 0.0613
Epoch 1/5, Loss: 0.0636
Epoch 1/5, Loss: 0.0650
Epoch 1/5, Loss: 0.0660
Epoch 1/5, Loss: 0.0672
Epoch 1/5, Loss: 0.0680
Epoch 1/5, Loss: 0.0688
Epoch 1/5, Loss: 0.0696
Epoch 1/5, Loss: 0.0703
Epoch 1/5, Loss: 0.0708
Epoch 1/5, Loss: 0.0712
Epoch 1/5, Loss: 0.0719
Epoch 1/5, Loss: 0.0726
Epoch 1/5, Loss: 0.0730
Epoch 1/5, Loss: 0.0732
Epoch 1/5, Loss: 0.0736
Epoch 1/5, Loss: 0.0738
Epoch 1/5, Loss: 0.0744
Epoch 1/5, Loss: 0.0747
Epoch 1/5, Loss: 0.0750
Epoch 1/5, Loss: 0.0753
Epoch 1/5, Loss: 0.0756
Epoch 1/5, Loss: 0.0757
Epoch 1/5, Loss: 0.0760
Epoch 1/5, Loss: 0.0761
Epoch 1/5, Loss: 0.0764
Epoch 1/5, Loss: 0.0768
Epoch 1/5, Loss: 0.0769
Epoch 1/5, Loss: 0.0771
Epoch 1/5, Loss:

 20%|██        | 1/5 [01:18<05:12, 78.14s/it]

Epoch 1/5, Loss: 0.2195
Epoch 1/5, Loss: 0.2195
Finished epoch0,latest loss0.0026943455450236797
Epoch 2/5, Loss: 0.0000
Epoch 2/5, Loss: 0.0000
Epoch 2/5, Loss: 0.0001
Epoch 2/5, Loss: 0.0001
Epoch 2/5, Loss: 0.0001
Epoch 2/5, Loss: 0.0001
Epoch 2/5, Loss: 0.0001
Epoch 2/5, Loss: 0.0002
Epoch 2/5, Loss: 0.0002
Epoch 2/5, Loss: 0.0003
Epoch 2/5, Loss: 0.0003
Epoch 2/5, Loss: 0.0004
Epoch 2/5, Loss: 0.0004
Epoch 2/5, Loss: 0.0004
Epoch 2/5, Loss: 0.0004
Epoch 2/5, Loss: 0.0005
Epoch 2/5, Loss: 0.0006
Epoch 2/5, Loss: 0.0006
Epoch 2/5, Loss: 0.0006
Epoch 2/5, Loss: 0.0006
Epoch 2/5, Loss: 0.0007
Epoch 2/5, Loss: 0.0007
Epoch 2/5, Loss: 0.0007
Epoch 2/5, Loss: 0.0007
Epoch 2/5, Loss: 0.0008
Epoch 2/5, Loss: 0.0009
Epoch 2/5, Loss: 0.0010
Epoch 2/5, Loss: 0.0010
Epoch 2/5, Loss: 0.0011
Epoch 2/5, Loss: 0.0011
Epoch 2/5, Loss: 0.0011
Epoch 2/5, Loss: 0.0012
Epoch 2/5, Loss: 0.0012
Epoch 2/5, Loss: 0.0012
Epoch 2/5, Loss: 0.0012
Epoch 2/5, Loss: 0.0012
Epoch 2/5, Loss: 0.0012
Epoch 2/5, Loss

 40%|████      | 2/5 [02:34<03:51, 77.14s/it]

...

Epoch 3/5, Loss: 0.0576
Epoch 3/5, Loss: 0.0576
Epoch 3/5, Loss: 0.0576
Epoch 3/5, Loss: 0.0577
Epoch 3/5, Loss: 0.0577
Epoch 3/5, Loss: 0.0577
Finished epoch2,latest loss0.0007007652893662453
Epoch 4/5, Loss: 0.0000
Epoch 4/5, Loss: 0.0001
Epoch 4/5, Loss: 0.0001
Epoch 4/5, Loss: 0.0001
Epoch 4/5, Loss: 0.0001
Epoch 4/5, Loss: 0.0001
Epoch 4/5, Loss: 0.0001
Epoch 4/5, Loss: 0.0001
Epoch 4/5, Loss: 0.0001
Epoch 4/5, Loss: 0.0002
Epoch 4/5, Loss: 0.0002
Epoch 4/5, Loss: 0.0003
Epoch 4/5, Loss: 0.0003
Epoch 4/5, Loss: 0.0003
Epoch 4/5, Loss: 0.0003
Epoch 4/5, Loss: 0.0003
Epoch 4/5, Loss: 0.0003
Epoch 4/5, Loss: 0.0003
Epoch 4/5, Loss: 0.0004
Epoch 4/5, Loss: 0.0004
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0005
Epoch 4/5, Loss: 0.0007
Epoch 4/5, Loss

 80%|████████  | 4/5 [05:01<01:14, 74.76s/it]

Epoch 4/5, Loss: 0.0521
Epoch 4/5, Loss: 0.0522
Epoch 4/5, Loss: 0.0522
Epoch 4/5, Loss: 0.0522
Finished epoch3,latest loss0.004225993063300848
Epoch 5/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0001
Epoch 5/5, Loss: 0.0001
Epoch 5/5, Loss: 0.0001
Epoch 5/5, Loss: 0.0001
Epoch 5/5, Loss: 0.0002
Epoch 5/5, Loss: 0.0002
Epoch 5/5, Loss: 0.0002
Epoch 5/5, Loss: 0.0002
Epoch 5/5, Loss: 0.0002
Epoch 5/5, Loss: 0.0002
Epoch 5/5, Loss: 0.0002
Epoch 5/5, Loss: 0.0003
Epoch 5/5, Loss: 0.0003
Epoch 5/5, Loss: 0.0003
Epoch 5/5, Loss: 0.0004
Epoch 5/5, Loss: 0.0004
Epoch 5/5, Loss: 0.0004
Epoch 5/5, Loss: 0.0004
Epoch 5/5, Loss: 0.0004
Epoch 5/5, Loss: 0.0004
Epoch 5/5, Loss: 0.0005
Epoch 5/5, Loss: 0.0005
Epoch 5/5, Loss: 0.0005
Epoch 5/5, Loss: 0.0005
Epoch 5/5, Loss: 0.0005
Epoch 5/5, Loss: 0.0006
Epoch 5/5, Loss: 0.0006
Epoch 5/5, Loss: 0.0006
Epoch 5/5, Loss: 0.0006
Epoch 5/5, Loss:

100%|██████████| 5/5 [06:14<00:00, 74.89s/it]

Epoch 5/5, Loss: 0.0455
Epoch 5/5, Loss: 0.0455
Epoch 5/5, Loss: 0.0455
Epoch 5/5, Loss: 0.0455
Finished epoch4,latest loss0.010470119304955006


In [19]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found
